# Análisis Exploratorio y Transformación de Datos del Hospital

Este notebook está segmentado en dos partes principales:

1. **Exploración de los datos:** Análisis de la calidad, unicidad, valores nulos, distribución de los datos, validaciones de formato y cruces entre tablas.
2. **Limpieza y transformación:** Procesos para mejorar la calidad, estandarizar y preparar los datos para análisis posteriores, considerando los hallazgos del análisis exploratorio.

## Parte 1: Exploración de los Datos

En esta sección se realiza un análisis exploratorio de las tablas de pacientes y citas médicas, identificando problemas de calidad y patrones relevantes.

In [ ]:
# ==========================
# Carga de datos y visualización inicial
# ==========================
# Este bloque carga los datos desde un archivo JSON y crea los DataFrames principales.
# Se visualizan las primeras filas para tener una idea general de la estructura y el contenido.

import json
import pandas as pd

with open('../dataset_original/dataset_hospital 2.json', encoding='utf-8') as f:
    data = json.load(f)

df_pacientes = pd.DataFrame(data['pacientes'])
df_citas = pd.DataFrame(data['citas_medicas']) if 'citas_medicas' in data and len(data['citas_medicas']) > 0 else pd.DataFrame()

print("Primeras filas de la tabla pacientes:")
display(df_pacientes.head())
print("\nPrimeras filas de la tabla citas_medicas:")
display(df_citas.head())

### 1.1 Análisis de la tabla de Pacientes

Se revisan tipos de datos, valores nulos, unicidad y distribución de los principales campos.

In [ ]:
# ==========================
# Análisis exploratorio de la tabla de pacientes
# ==========================
# Este bloque muestra información relevante sobre los tipos de datos, valores nulos,
# cantidad de valores únicos y la distribución de los principales campos de la tabla de pacientes.

print("Resumen de columnas, tipos y cantidad de datos faltantes:")
display(df_pacientes.info())
print("\nCantidad de datos faltantes por columna:")
display(df_pacientes.isnull().sum())
print("\nCantidad de valores únicos por columna:")
display(df_pacientes.nunique())

for col in df_pacientes.columns:
    print(f"\nColumna: {col}")
    print(f"Valores únicos: {df_pacientes[col].nunique(dropna=True)}")
    print("Principales valores:")
    display(df_pacientes[col].value_counts(dropna=False).head(10))
    print("-" * 40)

### 1.2 Análisis de la tabla de Citas Médicas

Se realiza el mismo análisis para la tabla de citas médicas.

In [ ]:
# ==========================
# Análisis exploratorio de la tabla de citas médicas
# ==========================
# Se revisan tipos de datos, valores nulos, unicidad y distribución de los principales campos de la tabla de citas.

if not df_citas.empty:
    print("Resumen de columnas, tipos y cantidad de datos faltantes:")
    display(df_citas.info())
    print("\nCantidad de datos faltantes por columna:")
    display(df_citas.isnull().sum())
    print("\nCantidad de valores únicos por columna:")
    display(df_citas.nunique())

    for col in df_citas.columns:
        print(f"\nColumna: {col}")
        print(f"Valores únicos: {df_citas[col].nunique(dropna=True)}")
        print("Principales valores:")
        display(df_citas[col].value_counts(dropna=False).head(10))
        print("-" * 40)
else:
    print("La tabla 'citas_medicas' está vacía o no contiene registros.")

### 1.3 Validaciones adicionales de formato y cruces entre tablas

En esta sección se realizan validaciones de formato y cruces entre las tablas para identificar problemas de integridad y coherencia antes de la transformación.

**Hallazgos principales:**
- 177 `id_paciente` en citas no existen en la tabla de pacientes. Ejemplo: [6146, 6148, 6149, 6150, 6157, 6159, 6673, 6164, 6676, 6169]
- 23 pacientes no tienen ninguna cita registrada.
- No hay citas con fecha anterior a la fecha de nacimiento del paciente.
- 640 citas de ginecología para pacientes no femeninas.
- 1601 citas con estado 'Completada' o 'Cancelada' pero sin fecha_cita.
- 1469 pacientes mayores de 18 años con citas en Pediatría.
- Todos los nombres de médicos tienen el prefijo esperado o están vacíos.
- Todas las especialidades están dentro del conjunto esperado o están vacías.
- Todos los costos son positivos o nulos.
- Todos los correos electrónicos tienen un formato válido.
- Todos los teléfonos tienen un formato válido o están vacíos.
- Todos los nombres tienen un formato válido.

### 1.4 Validaciones adicionales de formato en la tabla de pacientes

En este bloque se realizan validaciones adicionales sobre los datos de pacientes:
- Se valida que los correos electrónicos tengan un formato adecuado.
- Se verifica que los números de teléfono tengan el formato esperado.
- Se revisa que los nombres de los pacientes no contengan caracteres no permitidos o números.

In [ ]:
# ==========================
# Validaciones adicionales de formato en la tabla de pacientes
# ==========================

import re

# Validación de correos electrónicos
patron_email = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
if 'email' in df_pacientes.columns:
    emails = df_pacientes['email'].dropna()
    emails_invalidos = emails[~emails.str.match(patron_email)]
    if not emails_invalidos.empty:
        print(f"Se encontraron {len(emails_invalidos)} correos electrónicos con formato inválido:")
        display(emails_invalidos)
    else:
        print("Todos los correos electrónicos tienen un formato válido.")
else:
    print("No existe la columna 'email' en la tabla de pacientes.")

# Validación de formato de teléfono: debe ser 'XXX-XXX-XXXX' o vacío/nulo
patron_telefono = re.compile(r"^\d{3}-\d{3}-\d{4}$")
if 'telefono' in df_pacientes.columns:
    telefonos = df_pacientes['telefono'].dropna()
    telefonos_invalidos = telefonos[~telefonos.str.match(patron_telefono)]
    if not telefonos_invalidos.empty:
        print(f"Se encontraron {len(telefonos_invalidos)} teléfonos con formato inválido (esperado XXX-XXX-XXXX):")
        display(telefonos_invalidos)
    else:
        print("Todos los teléfonos tienen un formato válido o están vacíos.")
else:
    print("No existe la columna 'telefono' en la tabla de pacientes.")

# Validación de nombres: no deben contener números ni caracteres especiales (solo letras y espacios)
patron_nombre = re.compile(r"^[A-Za-zÁÉÍÓÚáéíóúÑñüÜ\s]+$")
if 'nombre' in df_pacientes.columns:
    nombres = df_pacientes['nombre'].dropna()
    nombres_invalidos = nombres[~nombres.str.match(patron_nombre)]
    if not nombres_invalidos.empty:
        print(f"Se encontraron {len(nombres_invalidos)} nombres con caracteres no permitidos:")
        display(nombres_invalidos)
    else:
        print("Todos los nombres tienen un formato válido.")
else:
    print("No existe la columna 'nombre' en la tabla de pacientes.")

In [ ]:
# ==========================
# Validaciones adicionales de formato en la tabla de citas médicas
# ==========================

# Validación de nombres de médicos: deben empezar con 'Dr.' o 'Dra.'
if 'medico' in df_citas.columns:
    medicos = df_citas['medico'].dropna()
    patron_medico = re.compile(r"^(Dr\.|Dra\.)\s")
    medicos_invalidos = medicos[~medicos.str.match(patron_medico)]
    if not medicos_invalidos.empty:
        print(f"Se encontraron {len(medicos_invalidos)} nombres de médicos que no empiezan con 'Dr.' o 'Dra.':")
        display(medicos_invalidos)
    else:
        print("Todos los nombres de médicos tienen el prefijo esperado o están vacíos.")
else:
    print("No existe la columna 'medico' en la tabla de citas.")

# Validación de especialidades: ejemplo de conjunto esperado (ajustar según el dataset real)
especialidades_esperadas = {
    "Cardiología", "Pediatría", "Neurología", "Ortopedia", "Ginecología"
}
if 'especialidad' in df_citas.columns:
    especialidades = df_citas['especialidad'].dropna()
    especialidades_invalidas = especialidades[~especialidades.isin(especialidades_esperadas)]
    if not especialidades_invalidas.empty:
        print(f"Se encontraron {len(especialidades_invalidas)} especialidades fuera del conjunto esperado:")
        display(especialidades_invalidas.value_counts())
    else:
        print("Todas las especialidades están dentro del conjunto esperado o están vacías.")
else:
    print("No existe la columna 'especialidad' en la tabla de citas.")

# Validación de costos: deben ser positivos o nulos
if 'costo' in df_citas.columns:
    costos = df_citas['costo'].dropna()
    costos_invalidos = costos[costos < 0]
    if not costos_invalidos.empty:
        print(f"Se encontraron {len(costos_invalidos)} costos negativos en citas médicas:")
        display(costos_invalidos)
    else:
        print("Todos los costos son positivos o nulos.")
else:
    print("No existe la columna 'costo' en la tabla de citas.")

In [ ]:
# ==========================
# Cruces y validaciones entre tablas
# ==========================

# 1. Validar que todos los id_paciente en citas existan en pacientes
if not df_citas.empty and 'id_paciente' in df_citas.columns and 'id_paciente' in df_pacientes.columns:
    ids_pacientes = set(df_pacientes['id_paciente'])
    ids_citas = set(df_citas['id_paciente'])
    ids_citas_sin_paciente = ids_citas - ids_pacientes
    if ids_citas_sin_paciente:
        print(f"Advertencia: Hay {len(ids_citas_sin_paciente)} id_paciente en citas que no existen en la tabla de pacientes.")
        print("Ejemplo de ids no encontrados:", list(ids_citas_sin_paciente)[:10])
    else:
        print("Todos los id_paciente de citas existen en la tabla de pacientes.")

    # 2. Validar que no haya pacientes sin ninguna cita
    pacientes_sin_cita = ids_pacientes - ids_citas
    print(f"Pacientes sin ninguna cita registrada: {len(pacientes_sin_cita)}")
else:
    print("No se puede validar id_paciente entre tablas por falta de columnas.")

# 3. Validar que las fechas de las citas sean posteriores a la fecha de nacimiento del paciente
import pandas as pd

def parse_fecha(fecha):
    """Convierte una fecha a Timestamp o NaT si no es válida."""
    try:
        return pd.to_datetime(fecha, errors='coerce')
    except Exception:
        return pd.NaT

if (
    not df_citas.empty and
    'fecha_cita' in df_citas.columns and
    'id_paciente' in df_citas.columns and
    'fecha_nacimiento' in df_pacientes.columns and
    'id_paciente' in df_pacientes.columns
):
    # Convertir las fechas a datetime para comparación segura
    df_citas_valid = df_citas.merge(
        df_pacientes[['id_paciente', 'fecha_nacimiento']],
        on='id_paciente',
        how='left'
    ).copy()
    df_citas_valid['fecha_cita_dt'] = df_citas_valid['fecha_cita'].apply(parse_fecha)
    # Asegurar que fecha_nacimiento es datetime
    if df_citas_valid['fecha_nacimiento'].dtype == object:
        df_citas_valid['fecha_nacimiento_dt'] = df_citas_valid['fecha_nacimiento'].apply(parse_fecha)
    else:
        df_citas_valid['fecha_nacimiento_dt'] = df_citas_valid['fecha_nacimiento']

    citas_antes_de_nacer = df_citas_valid[
        (df_citas_valid['fecha_cita_dt'].notnull()) &
        (df_citas_valid['fecha_nacimiento_dt'].notnull()) &
        (df_citas_valid['fecha_cita_dt'] < df_citas_valid['fecha_nacimiento_dt'])
    ]
    if not citas_antes_de_nacer.empty:
        print(f"Advertencia: Hay {len(citas_antes_de_nacer)} citas con fecha anterior a la fecha de nacimiento del paciente.")
        display(citas_antes_de_nacer[['id_cita', 'id_paciente', 'fecha_nacimiento', 'fecha_cita']].head())
    else:
        print("No hay citas con fecha anterior a la fecha de nacimiento del paciente.")
else:
    print("No se puede validar fechas de nacimiento vs. fechas de cita por falta de columnas.")

# 4. Validar que el sexo del paciente sea consistente con la especialidad de la cita (ejemplo: ginecología solo para mujeres)
if (
    not df_citas.empty and
    'especialidad' in df_citas.columns and
    'id_paciente' in df_citas.columns and
    'sexo' in df_pacientes.columns and
    'id_paciente' in df_pacientes.columns
):
    df_citas_valid = df_citas.merge(
        df_pacientes[['id_paciente', 'sexo']],
        on='id_paciente',
        how='left'
    )
    # Ejemplo: Ginecología solo para sexo femenino
    mask_gine = (
        df_citas_valid['especialidad'].str.lower() == 'ginecología'
    ) & (
        df_citas_valid['sexo'].notnull() & (df_citas_valid['sexo'] != 'Femenino')
    )
    inconsistentes_gine = df_citas_valid[mask_gine]
    if not inconsistentes_gine.empty:
        print(f"Advertencia: Hay {len(inconsistentes_gine)} citas de ginecología para pacientes no femeninas.")
        display(inconsistentes_gine[['id_cita', 'id_paciente', 'sexo', 'especialidad']].head())
    else:
        print("No hay inconsistencias de sexo en citas de ginecología.")
else:
    print("No se puede validar sexo vs. especialidad por falta de columnas.")

# 5. Validar que el costo de la cita sea positivo (si aplica)
if not df_citas.empty and 'costo' in df_citas.columns:
    citas_costo_negativo = df_citas[df_citas['costo'].notnull() & (df_citas['costo'] < 0)]
    if not citas_costo_negativo.empty:
        print(f"Advertencia: Hay {len(citas_costo_negativo)} citas con costo negativo.")
        display(citas_costo_negativo[['id_cita', 'id_paciente', 'costo']].head())
    else:
        print("No hay citas con costo negativo.")
else:
    print("No se puede validar costo de citas por falta de columna.")

# 6. Validar que el estado de la cita sea consistente con la presencia de fecha_cita
if not df_citas.empty and 'estado_cita' in df_citas.columns and 'fecha_cita' in df_citas.columns:
    # Ejemplo: una cita 'Completada' o 'Cancelada' debe tener fecha_cita no nula
    mask_estado = df_citas['estado_cita'].isin(['Completada', 'Cancelada'])
    inconsistentes_estado = df_citas[mask_estado & df_citas['fecha_cita'].isnull()]
    if not inconsistentes_estado.empty:
        print(f"Advertencia: Hay {len(inconsistentes_estado)} citas con estado 'Completada' o 'Cancelada' pero sin fecha_cita.")
        display(inconsistentes_estado[['id_cita', 'id_paciente', 'estado_cita', 'fecha_cita']].head())
    else:
        print("No hay inconsistencias entre estado_cita y fecha_cita.")
else:
    print("No se puede validar estado_cita vs. fecha_cita por falta de columnas.")

# 7. Validar que los pacientes que asisten a Pediatría no tengan más de 18 años
if (
    not df_citas.empty and
    'especialidad' in df_citas.columns and
    'id_paciente' in df_citas.columns and
    'edad' in df_pacientes.columns and
    'id_paciente' in df_pacientes.columns
):
    df_citas_ped = df_citas[df_citas['especialidad'].str.lower() == 'pediatría']
    if not df_citas_ped.empty:
        df_citas_ped = df_citas_ped.merge(
            df_pacientes[['id_paciente', 'edad']],
            on='id_paciente',
            how='left'
        )
        mayores_18 = df_citas_ped[df_citas_ped['edad'].notnull() & (df_citas_ped['edad'] > 18)]
        if not mayores_18.empty:
            print(f"Advertencia: Hay {len(mayores_18)} pacientes mayores de 18 años con citas en Pediatría.")
            display(mayores_18[['id_cita', 'id_paciente', 'edad', 'especialidad']].head())
        else:
            print("No hay pacientes mayores de 18 años con citas en Pediatría.")
    else:
        print("No hay citas registradas en la especialidad de Pediatría.")
else:
    print("No se puede validar edad de pacientes en Pediatría por falta de columnas.")

## Parte 2: Limpieza y Transformación de los Datos

En esta sección se aplican procesos de depuración, estandarización y transformación para preparar los datos, considerando los hallazgos encontrados en el análisis exploratorio y los cruces.

### 2.1 Estrategias de limpieza y transformación

**Pacientes:**
- Eliminar duplicados por id_paciente.
- Estandarizar capitalización de nombres y ciudades.
- Validar y corregir formato de fecha_nacimiento.
- Calcular edad si es posible.
- Estandarizar y completar valores en sexo.
- Rellenar nulos en email y teléfono con string vacío.
- Rellenar nulos en ciudad con 'Desconocido'.

**Citas médicas:**
- Corregir fechas inválidas y nulos en fecha_cita.
- Estandarizar y completar especialidad y medico.
- Rellenar nulos en costo con 0.
- Estandarizar y completar estado_cita.

### 2.2 Limpieza y transformación de la tabla de pacientes

En este bloque se aplican las estrategias de limpieza y estandarización a la tabla de pacientes, asegurando la calidad y consistencia de los datos.

In [2]:
# ==========================
# Transformación y limpieza de la tabla de pacientes
# ==========================
# Este bloque realiza la depuración y estandarización de los datos de pacientes:
# - Combina información de pacientes duplicados por id_paciente.
# - Estandariza nombres y ciudades.
# - Valida y corrige fechas de nacimiento.
# - Calcula la edad si es posible.
# - Estandariza el campo sexo.
# - Normaliza el formato de teléfono y gestiona nulos en email y teléfono.

# 1. Completar información de pacientes con el mismo id_paciente
def combinar_filas(grupo):
    # Para cada columna, toma el primer valor no nulo encontrado
    return grupo.ffill().bfill().iloc[0]

df_pacientes = (
    df_pacientes.groupby('id_paciente', as_index=False)
    .apply(combinar_filas)
    .reset_index(drop=True)
)

df_pacientes.reset_index(drop=True, inplace=True)

# 2. Estandarizar capitalización de nombres y ciudades
if 'nombre' in df_pacientes.columns:
    df_pacientes['nombre'] = df_pacientes['nombre'].astype(str).str.title().str.strip()
if 'ciudad' in df_pacientes.columns:
    df_pacientes['ciudad'] = df_pacientes['ciudad'].replace({None: pd.NA, 'None': pd.NA})
    # Rellenar ciudad solo si hay una ciudad dominante para ese paciente, si no dejar nulo
    # (ya se combinó arriba, así que solo quedan nulos reales)
    df_pacientes['ciudad'] = df_pacientes['ciudad'].fillna('Sin información')

# 3. Validar y corregir formato de fecha_nacimiento
if 'fecha_nacimiento' in df_pacientes.columns:
    df_pacientes['fecha_nacimiento'] = pd.to_datetime(df_pacientes['fecha_nacimiento'], errors='coerce')
    # Eliminar/corregir fechas de nacimiento mayores a la fecha actual
    hoy = pd.Timestamp.today().normalize()
    df_pacientes.loc[df_pacientes['fecha_nacimiento'] > hoy, 'fecha_nacimiento'] = pd.NaT

# 4. Calcular edad si es posible
from datetime import datetime
def calcular_edad(fecha_nac):
    if pd.isnull(fecha_nac):
        return None
    hoy = pd.Timestamp.today()
    return hoy.year - fecha_nac.year - ((hoy.month, hoy.day) < (fecha_nac.month, fecha_nac.day))

if 'edad' in df_pacientes.columns and 'fecha_nacimiento' in df_pacientes.columns:
    df_pacientes['edad'] = df_pacientes.apply(
        lambda row: calcular_edad(row['fecha_nacimiento']) if pd.isnull(row['edad']) and pd.notnull(row['fecha_nacimiento']) else row['edad'],
        axis=1
    )
    # Si sigue faltando edad, dejar nulo (no inventar ni poner 0)

# 5. Estandarizar y completar valores en sexo
if 'sexo' in df_pacientes.columns:
    df_pacientes['sexo'] = df_pacientes['sexo'].replace({
        'F': 'Femenino', 'f': 'Femenino', 'Female': 'Femenino',
        'M': 'Masculino', 'm': 'Masculino', 'Male': 'Masculino',
        None: pd.NA, 'None': pd.NA
    })
    # Si falta sexo, dejar nulo (no asumir "Desconocido" si no hay info)

# 6. Rellenar nulos en email y teléfono
if 'email' in df_pacientes.columns:
    # Email: si falta, dejar nulo (no inventar ni poner string vacío)
    df_pacientes['email'] = df_pacientes['email'].replace({None: pd.NA, 'None': pd.NA})

if 'telefono' in df_pacientes.columns:
    # Teléfono: si falta, dejar nulo (no poner string vacío)
    df_pacientes['telefono'] = df_pacientes['telefono'].replace({None: pd.NA, 'None': pd.NA})

    # Estandarizar formato de teléfono: dejar solo números y, si tiene 10 dígitos, formatear como 'XXX-XXX-XXXX'
    import re
    def normalizar_telefono(tel):
        if pd.isnull(tel):
            return pd.NA
        # Eliminar todo lo que no sea dígito
        solo_numeros = re.sub(r'\D', '', str(tel))
        if len(solo_numeros) == 10:
            return f"{solo_numeros[:3]}-{solo_numeros[3:6]}-{solo_numeros[6:]}"
        else:
            # Si no tiene 10 dígitos, dejar solo los números
            return solo_numeros if solo_numeros else pd.NA

    df_pacientes['telefono'] = df_pacientes['telefono'].apply(normalizar_telefono)

print("Pacientes limpios y transformados:")
display(df_pacientes.head())

NameError: name 'df_pacientes' is not defined

### 2.3 Limpieza y transformación de la tabla de citas médicas

En este bloque se aplican las estrategias de limpieza y estandarización a la tabla de citas médicas, asegurando la calidad y consistencia de los datos.

In [ ]:
# ==========================
# Transformación y limpieza de la tabla de citas médicas
# ==========================
# Este bloque realiza la depuración y estandarización de los datos de citas médicas
# considerando los hallazgos de la exploración y validaciones previas.

# 1. Eliminar citas cuyo id_paciente no existe en la tabla de pacientes
if 'id_paciente' in df_citas.columns and 'id_paciente' in df_pacientes.columns:
    ids_validos = set(df_pacientes['id_paciente'])
    df_citas = df_citas[df_citas['id_paciente'].isin(ids_validos)]

# 2. Corregir fechas inválidas y nulos en fecha_cita
if 'fecha_cita' in df_citas.columns:
    df_citas['fecha_cita'] = pd.to_datetime(df_citas['fecha_cita'], errors='coerce')

# 3. Estandarizar y completar especialidad y medico
if 'especialidad' in df_citas.columns:
    df_citas['especialidad'] = df_citas['especialidad'].replace({None: pd.NA, 'None': pd.NA})
    # Estandarizar capitalización
    df_citas['especialidad'] = df_citas['especialidad'].astype(str).str.title().str.strip()
    # Dejar nulo si no está en el conjunto esperado
    especialidades_esperadas = {"Cardiología", "Pediatría", "Neurología", "Ortopedia", "Ginecología"}
    df_citas.loc[~df_citas['especialidad'].isin(especialidades_esperadas), 'especialidad'] = pd.NA

if 'medico' in df_citas.columns:
    # Estandarizar nombres de médicos: solo dejar los que empiezan con 'Dr.' o 'Dra.'
    patron_medico = re.compile(r"^(Dr\.|Dra\.)\s")
    df_citas['medico'] = df_citas['medico'].where(df_citas['medico'].dropna().str.match(patron_medico), pd.NA)

# 4. Rellenar nulos en costo con 0 y eliminar/corregir costos negativos
if 'costo' in df_citas.columns:
    df_citas['costo'] = pd.to_numeric(df_citas['costo'], errors='coerce')
    df_citas['costo'] = df_citas['costo'].fillna(0)
    df_citas.loc[df_citas['costo'] < 0, 'costo'] = 0

# 5. Estandarizar y completar estado_cita
if 'estado_cita' in df_citas.columns:
    df_citas['estado_cita'] = df_citas['estado_cita'].replace({None: pd.NA, 'None': pd.NA})
    df_citas['estado_cita'] = df_citas['estado_cita'].astype(str).str.title().str.strip()
    # Dejar nulo si no es un estado esperado
    estados_esperados = {"Completada", "Cancelada", "Pendiente"}
    df_citas.loc[~df_citas['estado_cita'].isin(estados_esperados), 'estado_cita'] = pd.NA

# 6. Eliminar o reportar citas de ginecología para pacientes no femeninas
if 'especialidad' in df_citas.columns and 'id_paciente' in df_citas.columns and 'sexo' in df_pacientes.columns:
    df_citas = df_citas.merge(df_pacientes[['id_paciente', 'sexo']], on='id_paciente', how='left', suffixes=('', '_pac'))
    mask_gine = (df_citas['especialidad'] == 'Ginecología') & (df_citas['sexo'].notnull()) & (df_citas['sexo'] != 'Femenino')
    df_citas = df_citas[~mask_gine]
    df_citas = df_citas.drop(columns=['sexo'])

# 7. Eliminar o reportar citas de pediatría para pacientes mayores de 18 años
if 'especialidad' in df_citas.columns and 'id_paciente' in df_citas.columns and 'edad' in df_pacientes.columns:
    df_citas = df_citas.merge(df_pacientes[['id_paciente', 'edad']], on='id_paciente', how='left', suffixes=('', '_pac'))
    mask_ped = (df_citas['especialidad'] == 'Pediatría') & (df_citas['edad'].notnull()) & (df_citas['edad'] > 18)
    df_citas = df_citas[~mask_ped]
    df_citas = df_citas.drop(columns=['edad'])

# 8. Eliminar citas con estado 'Completada' o 'Cancelada' pero sin fecha_cita
if 'estado_cita' in df_citas.columns and 'fecha_cita' in df_citas.columns:
    mask_estado = df_citas['estado_cita'].isin(['Completada', 'Cancelada'])
    df_citas = df_citas[~(mask_estado & df_citas['fecha_cita'].isnull())]

# 9. Eliminar citas con fecha anterior a la fecha de nacimiento del paciente
if 'fecha_cita' in df_citas.columns and 'id_paciente' in df_citas.columns and 'fecha_nacimiento' in df_pacientes.columns:
    df_citas = df_citas.merge(df_pacientes[['id_paciente', 'fecha_nacimiento']], on='id_paciente', how='left', suffixes=('', '_pac'))
    mask_fecha = (df_citas['fecha_cita'].notnull()) & (df_citas['fecha_nacimiento'].notnull()) & (df_citas['fecha_cita'] < df_citas['fecha_nacimiento'])
    df_citas = df_citas[~mask_fecha]
    df_citas = df_citas.drop(columns=['fecha_nacimiento'])

print("Citas médicas limpias y transformadas:")
display(df_citas.head())

### 2.4 Indicadores de calidad antes y después de la limpieza

En este bloque se muestran indicadores clave de calidad de los datos antes y después de aplicar los procesos de limpieza y transformación. Esto permite comparar el impacto de las acciones realizadas sobre los datos.

In [ ]:
# ==========================
# Indicadores de calidad antes y después de la limpieza
# ==========================

# Recargar los datos originales para comparación justa
def cargar_dfs_raw():
    import json
    import pandas as pd
    with open('../dataset_original/dataset_hospital 2.json', encoding='utf-8') as f:
        data = json.load(f)
    df_pacientes_raw = pd.DataFrame(data['pacientes'])
    df_citas_raw = pd.DataFrame(data['citas_medicas']) if 'citas_medicas' in data and len(data['citas_medicas']) > 0 else pd.DataFrame()
    return df_pacientes_raw, df_citas_raw

df_pacientes_raw, df_citas_raw = cargar_dfs_raw()

print('--- Indicadores de calidad ANTES de la limpieza ---')
print('Pacientes:')
print('Total registros:', len(df_pacientes_raw))
print('Duplicados por id_paciente:', df_pacientes_raw.duplicated('id_paciente').sum())
print('Nulos por columna:')
display(df_pacientes_raw.isnull().sum())
print('Citas médicas:')
print('Total registros:', len(df_citas_raw))
print('Duplicados por id_cita:', df_citas_raw.duplicated('id_cita').sum())
print('Nulos por columna:')
display(df_citas_raw.isnull().sum())

print('--- Indicadores de calidad DESPUÉS de la limpieza ---')
print('Pacientes:')
print('Total registros:', len(df_pacientes))
print('Duplicados por id_paciente:', df_pacientes.duplicated('id_paciente').sum())
print('Nulos por columna:')
display(df_pacientes.isnull().sum())
print('Citas médicas:')
print('Total registros:', len(df_citas))
print('Duplicados por id_cita:', df_citas.duplicated('id_cita').sum())
print('Nulos por columna:')
display(df_citas.isnull().sum())

### 2.5 Exportación de los datos limpios

En este bloque se exportan los DataFrames limpios a archivos CSV y JSON para su uso posterior en análisis o carga a sistemas externos.

In [ ]:
# ==========================
# Exportación de datos limpios
# ==========================

# Exportar a CSV y JSON
# Pacientes
try:
    df_pacientes.to_csv('./datasets_limpios/pacientes_limpio.csv', index=False)
    df_pacientes.to_json('./datasets_limpios/pacientes_limpio.json', orient='records', force_ascii=False)
    print('Exportación de pacientes limpia exitosa.')
except Exception as e:
    print('Error exportando pacientes:', e)

# Citas médicas
try:
    df_citas.to_csv('./datasets_limpios/citas_medicas_limpio.csv', index=False)
    df_citas.to_json('./datasets_limpios/citas_medicas_limpio.json', orient='records', force_ascii=False)
    print('Exportación de citas médicas limpia exitosa.')
except Exception as e:
    print('Error exportando citas médicas:', e)

### 2.6 Comparación visual de indicadores de calidad antes y después de la limpieza

En este bloque se generan tablas comparativas y gráficos profesionales de los indicadores de calidad antes y después de la limpieza para pacientes y citas médicas.

Se utilizan gráficos de barras para mostrar de manera clara y visual las diferencias en los indicadores clave, como el total de registros y la cantidad de duplicados, antes y después de aplicar los procesos de limpieza y transformación.

In [ ]:
# Comparación visual de indicadores de calidad antes y después de la limpieza
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pacientes
pacientes_comp = pd.DataFrame({
    'Indicador': ['Total registros', 'Duplicados por id_paciente'] + [f'Nulos en {col}' for col in df_pacientes_raw.columns],
    'Antes': [
        len(df_pacientes_raw),
        df_pacientes_raw.duplicated('id_paciente').sum()
    ] + list(df_pacientes_raw.isnull().sum()),
    'Después': [
        len(df_pacientes),
        df_pacientes.duplicated('id_paciente').sum()
    ] + list(df_pacientes.isnull().sum())
})

# Citas médicas
citas_comp = pd.DataFrame({
    'Indicador': ['Total registros', 'Duplicados por id_cita'] + [f'Nulos en {col}' for col in df_citas_raw.columns],
    'Antes': [
        len(df_citas_raw),
        df_citas_raw.duplicated('id_cita').sum()
    ] + list(df_citas_raw.isnull().sum()),
    'Después': [
        len(df_citas),
        df_citas.duplicated('id_cita').sum()
    ] + list(df_citas.isnull().sum())
})

# Mostrar tablas comparativas
print('Comparativo de indicadores - Pacientes')
display(pacientes_comp)
print('Comparativo de indicadores - Citas médicas')
display(citas_comp)

# Visualización profesional con seaborn (solo para los totales y duplicados)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.barplot(
    data=pacientes_comp.iloc[:2].melt(id_vars='Indicador', var_name='Estado', value_name='Valor'),
    x='Indicador', y='Valor', hue='Estado', ax=axes[0]
)
axes[0].set_title('Pacientes: Total y Duplicados')
axes[0].set_ylabel('Cantidad')
axes[0].set_xlabel('Indicador')

sns.barplot(
    data=citas_comp.iloc[:2].melt(id_vars='Indicador', var_name='Estado', value_name='Valor'),
    x='Indicador', y='Valor', hue='Estado', ax=axes[1]
)
axes[1].set_title('Citas médicas: Total y Duplicados')
axes[1].set_ylabel('Cantidad')
axes[1].set_xlabel('Indicador')

plt.tight_layout()
plt.show()

# Supuestos adoptados y recomendaciones de mejora

A continuación se detallan de manera más extensa y organizada los supuestos adoptados durante el análisis, limpieza y transformación de los datos:

## 1. Supuestos sobre valores nulos y datos faltantes
- **Campos críticos:** Se considera que los valores nulos en campos críticos como `id_paciente`, `fecha_nacimiento` y `especialidad` no deben ser rellenados con valores inventados ni por defecto. Estos registros se eliminan o se dejan como nulos para evitar introducir información errónea.
- **Campos no críticos:** En campos no críticos como `ciudad`, se puede rellenar el valor nulo con un string genérico como "Sin información" para facilitar el análisis posterior, pero siempre dejando constancia de que el dato original no estaba presente.
- **Email y teléfono:** Si faltan, se dejan como nulos y no se inventan ni se rellenan con cadenas vacías, para no falsear la información de contacto.

## 2. Supuestos sobre formatos y estandarización
- **Nombres de médicos:** Se asume que todos los nombres de médicos deben iniciar con el prefijo "Dr." o "Dra." para mantener la uniformidad y facilitar la identificación del profesional.
- **Especialidades:** Solo se aceptan especialidades dentro de un conjunto predefinido (Cardiología, Pediatría, Neurología, Ortopedia, Ginecología). Cualquier valor fuera de este conjunto se considera inválido y se elimina o se deja nulo.
- **Teléfonos:** El formato válido es "XXX-XXX-XXXX". Si el número no cumple con este formato pero tiene 10 dígitos, se normaliza. Si no es posible normalizar, se deja nulo.
- **Correos electrónicos:** Deben cumplir con un patrón estándar de email. Los que no cumplen se consideran inválidos y se reportan.
- **Nombres de pacientes:** No deben contener números ni caracteres especiales, solo letras y espacios. Los nombres inválidos se reportan para su corrección manual.

## 3. Supuestos sobre integridad referencial y consistencia
- **Citas sin paciente válido:** Se eliminan todas las citas cuyo `id_paciente` no existe en la tabla de pacientes para asegurar la integridad referencial.
- **Pacientes sin citas:** Se reportan pero no se eliminan, ya que pueden ser pacientes nuevos o sin atención registrada aún.
- **Fechas:** Se asume que ninguna cita puede tener una fecha anterior a la fecha de nacimiento del paciente. Estos registros se eliminan.
- **Citas de especialidad y sexo:** Se asume que solo pacientes de sexo femenino pueden tener citas en Ginecología. Las citas que no cumplen se eliminan.
- **Citas de Pediatría:** Solo pacientes de 18 años o menos pueden tener citas en Pediatría. Las citas que no cumplen se eliminan.
- **Costos:** Todos los costos deben ser positivos o nulos. Los valores negativos se corrigen a 0.
- **Estado de la cita:** Si una cita está marcada como "Completada" o "Cancelada", debe tener una fecha de cita válida. Si no la tiene, se elimina.

## 4. Supuestos sobre la transformación y limpieza
- **Estandarización de capitalización:** Se estandarizan los nombres y ciudades a formato título para mantener uniformidad.
- **Cálculo de edad:** Si la edad no está presente pero existe la fecha de nacimiento, se calcula automáticamente. Si no es posible, se deja nulo.
- **Sexo:** Se estandarizan los valores a "Femenino" y "Masculino". Si no hay información, se deja nulo.

## 5. Supuestos sobre la exportación y uso posterior
- **Exportación:** Los datos limpios se exportan en formatos CSV y JSON para facilitar su reutilización en otros sistemas o análisis.
- **No se rellenan datos faltantes con valores por defecto** salvo en casos justificados (por ejemplo, costo=0 si está nulo).

## 6. Recomendaciones de mejora
- Implementar validaciones automáticas y monitoreo continuo de calidad en futuras cargas de datos.
- Mantener un diccionario de datos y reglas de validación documentadas para el equipo.
- Usar herramientas de validación como Great Expectations o pytest para pruebas automáticas.
- Realizar auditorías periódicas y establecer responsables de calidad de datos.

## Adicional: Reporte avanzado de calidad y exploración de datos

A continuación se genera un reporte profesional y exploratorio de los datos limpios usando ydata-profiling. El reporte HTML contendrá estadísticas, distribuciones, correlaciones y alertas de calidad para facilitar la revisión y documentación del dataset.

In [ ]:
# Llamado a la generación de reportes avanzados desde el script externo
import sys
import os
sys.path.append('./scripts')
from pruebas_automaticas import pruebas_automaticas

# Generar reportes avanzados para los datos limpios y originales
pruebas_automaticas('limpio')
pruebas_automaticas('original')

## Adicional: Simulación de carga de datos
A continuación se simula la carga de los DataFrames limpios a una base de datos SQL utilizando SQLAlchemy. Esto permite verificar que los datos se pueden insertar correctamente y que cumplen con las restricciones de integridad referencial y formato.

In [3]:
# Llamado a la migración simulada al final del notebook
import sys
sys.path.append('../scripts')
try:
    import migracion
    print('Migración simulada ejecutada correctamente.')
except Exception as e:
    print(f'Error al ejecutar la migración simulada: {e}')

Migración simulada ejecutada correctamente.
